# Road Following - 학습

앞에서 수집한 데이터(이미지, X좌표, Y좌표)로 PyTorch 딥러닝 모델 프레임워크를 이용 to ResNet18 모델을 이용하여 학습.

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### 데이터 준비

    - 수집한 데이터 준비

In [2]:
!unzip -q road_following_1.zip

### 데이터 인스턴트 생성

In [3]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### 훈련데이터, 테스트 데이터 준비

    - 학습데이터 90% 테스트 데이터 10%

In [4]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### 데이터 로더 생성

In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### 학습에 사용한 모델 

PyTorch TorchVision에서 사용 가능한 ResNet-18 모델을 사용

In [6]:
model = models.resnet18(pretrained=True)

In [7]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

In [8]:
NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

0.793736, 1.915384
0.174566, 0.796802
0.048120, 0.154529
0.027806, 0.058277
0.021765, 0.019635
0.016414, 0.031048
0.016815, 0.020944
0.016259, 0.017677
0.019669, 0.007992
0.014947, 0.011751
0.010254, 0.022828
0.013735, 0.017060
0.011752, 0.027115
0.013968, 0.014405
0.013165, 0.010810
0.008854, 0.008433
0.010383, 0.007772
0.006215, 0.014039
0.025788, 0.018528
0.020904, 0.016028
0.013412, 0.008811
0.010360, 0.013513
0.011877, 0.013456
0.012206, 0.006378
0.012450, 0.021733
0.012848, 0.009314
0.007086, 0.004878
0.006194, 0.005979
0.005997, 0.008073
0.009269, 0.006867
0.006192, 0.008050
0.005068, 0.005906
0.005428, 0.009348
0.005381, 0.007079
0.007746, 0.006386
0.005702, 0.005866
0.005236, 0.008839
0.005359, 0.004854
0.007948, 0.005127
0.003228, 0.005577
0.004276, 0.011765
0.004223, 0.003639
0.002494, 0.005169
0.002676, 0.011383
0.003225, 0.008020
0.002335, 0.005892
0.004293, 0.012782
0.005754, 0.008614
0.005728, 0.004932
0.005138, 0.004879
0.003842, 0.005541
0.003889, 0.006919
0.003554, 0.

- 학습이 완료되면 best_steering_model_xy.pth 모델 생성